In [3]:
###Import required packages
from models import loss_biological, p_aminostyrene, is_oscillatory
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import pandas as pd
from pyDOE import lhs

In [4]:
###Specify if data is to be saved to CSV
save_data = False

In [5]:
###Helper functions for naming
def name_converter(A):
    name = ''
    for i in range(3):
        arch = A[i]
        if arch == (1, 0, 0, 0, 0):
            name += 'A1'
        elif arch == (0, 1, 0, 0, 0):
            name += 'A2'
        elif arch == (0, 0, 1, 0, 0):
            name += 'R1'
        elif arch == (0, 0, 0, 1, 0):
            name += 'R2'
        elif arch == (0, 0, 0, 0, 1):
            name += 'N'
        name += '-'
    return name[:-1]

def name_converter_str(A):
    name = ''
    for i in range(3):
        arch = A[i]
        if arch == '(1, 0, 0, 0, 0)':
            name += 'A1'
        elif arch == '(0, 1, 0, 0, 0)':
            name += 'A2'
        elif arch == '(0, 0, 1, 0, 0)':
            name += 'R1'
        elif arch == '(0, 0, 0, 1, 0)':
            name += 'R2'
        elif arch == '(0, 0, 0, 0, 1)':
            name += 'N'
        name += '-'
    return name[:-1]

###Define search space
architecture = [hp.choice('prom1', [[0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), hp.choice('prom2', [[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), hp.choice('prom3', [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])]
space = (architecture, 
            [[hp.uniform('theta_paf_prom1', 1E-7, 10), hp.uniform('theta_paf_prom2', 1E-7, 10), hp.uniform('theta_paf_prom3', 1E-7, 10)], [hp.uniform('theta_paca_prom1', 1E-7, 10), hp.uniform('theta_paca_prom2', 1E-7, 10), hp.uniform('theta_paca_prom3', 1E-7, 10)]], 
            [[hp.uniform('k_paf_papA', 1E-7, 5), hp.uniform('k_paf_papB', 1E-7, 5), hp.uniform('k_paf_papC', 1E-7, 5), hp.uniform('k_paf_prom2', 1E-7, 5), hp.uniform('k_paf_prom3', 1E-7, 5)], [hp.uniform('k_paca_papA', 1E-7, 5), hp.uniform('k_paca_papB', 1E-7, 5), hp.uniform('k_paca_papC', 1E-7, 5), hp.uniform('k_paca_prom2', 1E-7, 5), hp.uniform('k_paca_prom3', 1E-7, 5)]])

global perturbs 
###Objective function
def run_hyperopt(max_iters):
    losses = []
    params = []
    circuits = []

    #Define objective function
    def objective(args):
        architecture, thetas, ks = args
        #Integration conditions
        t = np.linspace(0, 1.73E5, 200) 
        y0 = np.array([0., 0., 0., 0., 0. , 0., 0., 0., 0., 0., 0., 0. , 0., 0., 0., 0., 0., 0., 0. , 0., 0., 0., 0., 0., 0., 0., 0., 0.]) 
        extra_options = {'old_api': False, 'user_data': [architecture, thetas, ks, perturbs], 'rtol':1E-4}
        ode_solver = ode('cvode', p_aminostyrene, **extra_options)
        solution = ode_solver.solve(t, y0)
        j1, j2 = solution.values.y[-1, -2:]
        j1, j2, loss = loss_biological(j1, j2, alpha1=1E-12, alpha2=1E-7)
        loss += is_oscillatory(solution, beta = 10E3)
        losses.append(loss)
        params.append([thetas, ks])
        circuits.append(architecture)
        return loss

    #Run hyperopt call
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_iters)
    #Create trajectory data frame
    theta_paf_prom1s, theta_paf_prom2s, theta_paf_prom3s, theta_paca_prom1s, theta_paca_prom2s, theta_paca_prom3s = [[], [], [], [], [], []]
    k_paf_papAs, k_paf_papBs, k_paf_papCs, k_paf_prom2s, k_paf_prom3s, k_paca_papAs, k_paca_papBs, k_paca_papCs, k_paca_prom2s, k_paca_prom3s = [[], [], [], [], [], [], [], [], [], []]
    for i in range(len(params)):
        thetas = params[0][0]
        ks = params[0][1]
        theta_paf_prom1, theta_paf_prom2, theta_paf_prom3 = thetas[0]
        theta_paca_prom1, theta_paca_prom2, theta_paca_prom3 = thetas[1]
        k_paf_papA, k_paf_papB, k_paf_papC, k_paf_prom2, k_paf_prom3 = ks[0]
        k_paca_papA, k_paca_papB, k_paca_papC, k_paca_prom2, k_paca_prom3 = ks[1]

        theta_paf_prom1s.append(theta_paf_prom1)
        theta_paf_prom2s.append(theta_paf_prom2)
        theta_paf_prom3s.append(theta_paf_prom3)
        theta_paca_prom1s.append(theta_paca_prom1)
        theta_paca_prom2s.append(theta_paca_prom2)
        theta_paca_prom3s.append(theta_paca_prom3)
        k_paf_papAs.append(k_paf_papA)
        k_paf_papBs.append(k_paf_papB)
        k_paf_papCs.append(k_paf_papC)
        k_paf_prom2s.append(k_paf_prom2)
        k_paf_prom3s.append(k_paf_prom3)
        k_paca_papAs.append(k_paca_papA)
        k_paca_papBs.append(k_paca_papB)
        k_paca_papCs.append(k_paca_papC)
        k_paca_prom2s.append(k_paca_prom2)
        k_paca_prom3s.append(k_paca_prom3)

    landscape = pd.DataFrame({'circuit':circuits, 'loss': losses, 'theta_paf_prom1':theta_paf_prom1s, 'theta_paf_prom2':theta_paf_prom2s, 'theta_paf_prom3':theta_paf_prom3s,
                'theta_paca_prom1':theta_paca_prom1s, 'theta_paca_prom2':theta_paca_prom2s, 'theta_paca_prom3':theta_paca_prom3,
                'k_paf_papA':k_paf_papAs, 'k_paf_papB':k_paf_papBs, 'k_paf_papC':k_paf_papCs, 'k_paf_prom2':k_paf_prom2s, 'k_paf_prom3':k_paf_prom3s,
                'k_paca_papA':k_paca_papAs, 'k_paca_papB':k_paca_papBs, 'k_paca_papC':k_paca_papCs, 'k_paca_prom2':k_paca_prom2s, 'k_paca_prom3':k_paca_prom3s})

    best_loss = 1E5
    best_circuit = 'Initial'
    best_losses = []
    best_losses_circuits = []
    for i in range(len(landscape)):
        if landscape.loss[i] < best_loss:
            best_loss = landscape.loss[i]
            best_circuit = landscape.circuit[i]
        best_losses.append(best_loss)
        best_losses_circuits.append(best_circuit)
    landscape['best_losses'] = best_losses
    landscape['best_loss_circuit'] = best_losses_circuits
    landscape['perturb1'] = perturbs[0]
    landscape['perturb0'] = perturbs[1]

    landscape['Circuit'] = [name_converter(landscape.circuit[i]) for i in range(len(landscape))]
    landscape['Best Circuit'] = [name_converter(landscape.best_loss_circuit[i]) for i in range(len(landscape))]
    landscape = landscape.reset_index()
    
    return landscape, best

In [ ]:
###Run sample optimization
perturbs= [5E-4, 5E1]
max_iters = 1000
landscape, best = run_hyperopt(max_iters)
if save_data: landscape.to_csv('../data/p_aminostyrene_sample_run.csv')

In [ ]:
##Assess background by running 100x
max_iters = 1000
total_background = pd.DataFrame()
perturbs= [5E-4, 5E1]
for i in range(100):
    background, best = run_hyperopt(max_iters)
    total_background = pd.concat([total_background, background])

if save_data: total_background.to_csv('../data/p_aminostyrene_background.csv')


In [ ]:
###Chemical robustness experiments
global perturbs
max_iters = 1000
total_landscape = pd.DataFrame()
total_perturbs = lhs(2, samples=100)
total_perturbs = total_perturbs*([1E-3 - 1E-4, 1E2 - 1E1]) + [1E-4, 1E1]

for p in total_perturbs:
    perturbs = p
    landscape, best = run_hyperopt(max_iters)
    landscape.to_csv('chemical_robustness.csv', mode='a', header=False)
    total_landscape = pd.concat([total_landscape, landscape])

if save_data: total_landscape.to_csv('../data/p_aminostyrene_chemical_robustness.csv')